In [1]:
import os 
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
# Load enviroment variables in a file called .env
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key 
if not api_key:
    print("No API key was found")
else:
    print("API key found")

openai = OpenAI()

API key found


In [3]:
# Defining the Website class
class Website:
    
    # Declaration of the expected attributes for the Website class
    url: str
    title: str 
    text: str

    # The __init__ method is the constructor of the class, called when a new instance is created
    def __init__(self, url):
        # Initializes the 'url' attribute with the value passed as an argument
        self.url = url
        
        # Makes an HTTP request to get the content of the URL
        response = requests.get(url)
        
        # Uses BeautifulSoup to parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extracts the page's title. If there's no title, set it to "No title found"
        self.title = soup.title.string if soup.title else "No title found"
        
        # Removes irrelevant elements from the page (such as scripts, images, and inputs)
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        
        # Extracts the page's text, separating lines by newlines and stripping extra spaces
        self.text = soup.body.get_text(separator="\n", strip=True)

In [4]:
# "You are an assistant that analyzes the contents of a website": O assistente deve analisar o conteúdo de um site.
# "and provides a short summary": O assistente deve fornecer um resumo curto do conteúdo.
# "ignoring text that might be navigation related": O assistente deve ignorar textos que sejam relacionados à navegação, como menus, links de navegação, etc.
# "respond in markdown": O assistente deve responder usando a formatação Markdown, que é uma maneira de formatar texto simples, muito comum em plataformas como GitHub e Reddit.
system_prompt ="You are an assistant that analyzes the contents of a website\
                and provides a short summary, ignoring text that might be navigation related\
                respond in markdown.\
                You always answer in Portuguese (Brazilian Portuguese)"

In [5]:
def user_prompt_for(website):
    # Cria a variável 'user_prompt' e começa a mensagem com o título do site
    user_prompt = f"You are looking at a website title {website.title}"
    
    # Adiciona uma mensagem sobre o conteúdo do site e as instruções para resumir
    user_prompt += "\nThe contents of this website is as follows;\
 please provide a short summary of this website in markdown\
 if it includes news or annoucements, then summarize these too\n\n"
        
    # Adiciona o conteúdo textual extraído do site ao prompt
    user_prompt += website.text
    
    # Retorna o prompt final, que será usado pelo modelo de IA
    return user_prompt

In [6]:
def messages_for(website):
    return[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content":user_prompt_for(website)},
    ]

In [7]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [8]:
def display_summary(url): 
    summary = summarize(url)
    display(Markdown(summary))

In [9]:
display_summary("https://g1.globo.com/globonews/")

# Resumo do site GloboNews

GloboNews é um canal de notícias que opera 24 horas, trazendo as últimas informações e eventos importantes tanto do Brasil quanto do mundo.

## Principais notícias

- **Rússia e Ucrânia**: Autoridades russas afirmam ter tomado a cidade de **Zelene**, localizada na região de Donetsk.
  
- **Fortes Chuvas em São Paulo**: Uma pessoa morreu devido às intensas chuvas; o corpo do artista **Rodolpho Tamanini Netto** foi encontrado.

- **Conflito Israel-Palestina**: 
  - Israel soltou prisioneiros palestinos na Cisjordânia.
  - O Hamas declarou que um refém israelense civil, que não foi libertada recentemente, está viva.
  - O Hamas também libertou quatro reféns israelenses.

- **Aeroporto Galeão**: O número de passageiros no aeroporto do Rio de Janeiro cresceu em **82%** em 2024, totalizando mais de **14 milhões** de pessoas. 

Essas informações refletem a diversidade de coberturas e o foco em eventos relevantes atuais.